In [1]:
import os
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
sys.path.append('/Users/aghavamp/Desktop/Projects/SUITPy')
import getpass
import importlib
import tqdm

import scipy.io as sio
import rsatoolbox as rsa
from rsatoolbox.io import spm as spm_io
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import surfAnalysisPy as surf
import SUITPy as suit
import nibabel as nb
from nibabel import cifti2
import nitools as nt
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from pathlib import Path
import seaborn as sns
import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel
import glob
import matplotlib.patches as patches
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'
analysisDir = os.path.join(os.path.dirname(os.path.dirname(baseDir)), 'analysis')



## wb_view subject space

In [21]:
import subprocess

sn = 101

names = ['lhand', 'rhand', 'bi', 'mvpa_rhand', 'mvpa_lhand', 'mvpa_bi']
hem = ['L', 'R']

# add paths:
paths = []
# add surface files:
for h in hem:
    paths.append(os.path.abspath(os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.inflated.32k.surf.gii')))

# add searchlight results:
for name in names:
    paths.append(os.path.abspath(os.path.join(baseDir, 'searchlight', f'{name}_s{sn}_cortex.dscalar.nii')))


# Run wb_view:
subprocess.run(['wb_view', *paths])





Info: Resources loaded:
   :/About   :/BalsaUploadDialog   :/Cursor   :/DingOntology   :/Fonts   :/general_resources.qrc   :/help_resources.qrc   :/HelpFiles   :/LayersPanel   :/MessageDialog   :/PaletteEditorDialog   :/PaletteSettings   :/qpdf   :/qt-project.org   :/RecentFilesDialog   :/SceneFileDialog   :/SpecFileDialog   :/Splash   :/ToolBar   :/update_resources.sh


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.L.inflated.32k.surf.gii was 0.027731 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.R.inflated.32k.surf.gii was 0.026955 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/mvpa_rhand_s101_cortex.dscalar.nii was 0.028202 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/mvpa_lhand_s101_cortex.dscalar.nii was 0.023151 seconds.


Info: Time to read /Users/aghavamp/Desk

CompletedProcess(args=['wb_view', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.L.inflated.32k.surf.gii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.R.inflated.32k.surf.gii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/lhand_s101_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/rhand_s101_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/bi_s101_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/mvpa_rhand_s101_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/mvpa_lhand_s101_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/mvpa_bi_s101_cortex.dscalar.nii'], returncode=0)

## surf plot MPVA

In [14]:
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
cmap = 'jet'
dmax = 0.1
searchlight_names = ['mvpa_lhand', 'mvpa_rhand', 'mvpa_bi']

borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
           'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}

for _, sn in enumerate(participants):
    for searchlight_name in searchlight_names: 
        Hem = ['L', 'R']
        D = {'L': np.zeros((32492)), 'R': np.zeros((32492))}
        for h in Hem:
            file = os.path.join(baseDir, 'searchlight', f'{searchlight_name}_s{sn}_{h}_cortex.dscalar.nii')  
            cifti_img = nb.load(file)
            data = cifti_img.get_fdata()
            header = cifti_img.header
            scalar_axis = header.get_axis(1)
            vertex_indices = scalar_axis.vertex

            D[h][vertex_indices] = data[0]
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        vmax = dmax #np.nanpercentile(all_data, dmax)
        vmin = -vmax
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H], f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap=cmap,
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap=cmap)
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('avg distance')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'searchlight {searchlight_name}, s{sn}')
        fig.tight_layout()
        plt.savefig(f'../figures/glmsingle/searchlight/{searchlight_name}_s{sn}.pdf', bbox_inches="tight")
        plt.close()



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/3283242395.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/3283242395.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/3283242395.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/3283242395.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/3283242395.py:52: UserWarning: This figure includes Axes that are n

## surf plot sanity (tmap)

In [8]:
participants = [101,102,104,103,106,107,108,109,110,111,112,113,114,115]
cmap = 'jet'
tmax = 99.9
searchlight_names = ['lhand', 'rhand', 'bi']

borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
           'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}

for searchlight_name in searchlight_names:
    for _, sn in enumerate(participants):
        Hem = ['L', 'R']
        D = {'L': np.zeros((32492)), 'R': np.zeros((32492))}
        for h in Hem:
            file = os.path.join(baseDir, 'searchlight', f'{searchlight_name}_s{sn}_{h}_cortex.dscalar.nii')  
            cifti_img = nb.load(file)
            data = cifti_img.get_fdata()
            header = cifti_img.header
            scalar_axis = header.get_axis(1)
            vertex_indices = scalar_axis.vertex

            D[h][vertex_indices] = data[0]
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        vmax = np.nanpercentile(all_data, tmax)
        vmin = -vmax
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H], f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap=cmap,
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap=cmap)
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('t-statistic')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'searchlight {searchlight_name}, s{sn}')
        fig.tight_layout()
        plt.savefig(f'../figures/glmsingle/searchlight/{searchlight_name}_s{sn}.pdf', bbox_inches="tight")
        plt.close()



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/2726364352.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/2726364352.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/2726364352.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/2726364352.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_42499/2726364352.py:52: UserWarning: This figure includes Axes that are n

## Concat subjects

In [20]:
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
searchlight_names = ['mvpa_lhand', 'mvpa_rhand', 'mvpa_bi']

borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
           'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}

for _, sn in enumerate(participants):
    for searchlight_name in searchlight_names: 
        left_img = nb.load(os.path.join(baseDir, 'searchlight', f'{searchlight_name}_s{sn}_L_cortex.dscalar.nii'))
        right_img = nb.load(os.path.join(baseDir, 'searchlight', f'{searchlight_name}_s{sn}_R_cortex.dscalar.nii'))

        # Get axes - for dscalar: axis 0 is ScalarAxis, axis 1 is BrainModelAxis
        left_scalar_axis = left_img.header.get_axis(0)  # ScalarAxis
        right_scalar_axis = right_img.header.get_axis(0)  # ScalarAxis
        
        left_brain_axis = left_img.header.get_axis(1)  # BrainModelAxis (spatial)
        right_brain_axis = right_img.header.get_axis(1)  # BrainModelAxis (spatial)

        # Verify scalar axes are compatible (same number of scalars/maps)
        if left_scalar_axis != right_scalar_axis:
            print("Warning: ScalarAxis differ between hemispheres")
            # You might want to handle this case specifically

        # Get data
        left_data = left_img.get_fdata()
        right_data = right_img.get_fdata()

        # Concatenate BrainModelAxis (spatial axis)
        combined_brain_axis = left_brain_axis + right_brain_axis
        
        # Concatenate data along axis 1 (spatial dimension)
        combined_data = np.concatenate([left_data, right_data], axis=1)
        
        # Create new axes tuple: (ScalarAxis, combined BrainModelAxis)
        new_axes = (left_scalar_axis, combined_brain_axis)
        
        # Create new header and image
        new_header = cifti2.Cifti2Header.from_axes(new_axes)
        new_img = cifti2.Cifti2Image(combined_data, new_header)
        
        # Save
        nb.save(new_img, os.path.join(baseDir, 'searchlight', f'{searchlight_name}_s{sn}_cortex.dscalar.nii'))


def concatenate_cifti2_files(file_paths, output_path=None):
    """
    Concatenate multiple CIFTI2 files along the scalar axis (rows)
    
    Parameters:
    -----------
    file_paths : list
        List of paths to CIFTI2 files to concatenate
    output_path : str, optional
        Path to save the concatenated file
    
    Returns:
    --------
    cifti_img : nibabel.cifti2.Cifti2Image
        Concatenated CIFTI2 image
    """
    
    # Load all CIFTI files
    cifti_imgs = [nb.load(path) for path in file_paths]
    
    # Get data arrays
    data_arrays = [img.get_fdata() for img in cifti_imgs]
    
    # Concatenate along axis 0 (rows/scalar axis)
    concatenated_data = np.concatenate(data_arrays, axis=0)
    
    # Get the header from the first file as template
    template_header = cifti_imgs[0].header
    
    # Create new axes for the concatenated data
    # Keep the column axis (axis 1) from the first file
    col_axis = template_header.get_axis(1)
    
    # Create a new scalar axis for the concatenated rows
    from nibabel.cifti2 import cifti2_axes
    
    # If the original files have ScalarAxis, create a new one with combined length
    row_axis = template_header.get_axis(0)
    if isinstance(row_axis, cifti2_axes.ScalarAxis):
        # Create names for the new scalar axis
        new_names = []
        for i, img in enumerate(cifti_imgs):
            orig_axis = img.header.get_axis(0)
            if hasattr(orig_axis, 'name'):
                for j in range(orig_axis.size):
                    new_names.append(f"{orig_axis.name[j]}_file{i}")
            else:
                for j in range(orig_axis.size):
                    new_names.append(f"scalar_{j}_file{i}")
        
        new_row_axis = cifti2_axes.ScalarAxis(new_names)
    else:
        # For other axis types, you might need different handling
        new_row_axis = row_axis
    
    # Create new header
    new_header = nb.cifti2.Cifti2Header.from_axes((new_row_axis, col_axis))
    
    # Create new CIFTI2 image
    new_cifti = nb.cifti2.Cifti2Image(concatenated_data, header=new_header)
    
    # Save if output path is provided
    if output_path:
        nb.save(new_cifti, output_path)
    
    return new_cifti

# Usage example
file_paths = ['file1.dscalar.nii', 'file2.dscalar.nii', 'file3.dscalar.nii']
concatenated_cifti = concatenate_cifti2_files(file_paths, 'concatenated.dscalar.nii')

FileNotFoundError: No such file or no access: 'file1.dscalar.nii'